# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [5]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [6]:
# Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 7ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

Epoch 137/150
33/33 [==============================] - 0s 986us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 924us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=======

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [7]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [8]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [9]:
# Train the model
normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 909us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================]

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 137/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [10]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [11]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: 0.4594 - mse: 0.4594 - val_loss: 0.2200 - val_mse: 0.2200
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2282 - mse: 0.2282 - val_loss: 0.1756 - val_mse: 0.1756
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1872 - mse: 0.1872 - val_loss: 0.1911 - val_mse: 0.1911
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1790 - mse: 0.1790 - val_loss: 0.1447 - val_mse: 0.1447
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1533 - mse: 0.1533 - val_loss: 0.1436 - val_mse: 0.1436
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1415 - mse: 0.1415 - val_loss: 0.1359 - val_mse: 0.1359
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1263 - mse: 0.1263 - val_loss: 0.1310 - val_mse: 0.1310
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1213 - m

Epoch 63/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0244 - mse: 0.0244 - val_loss: 0.1312 - val_mse: 0.1312
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0241 - mse: 0.0241 - val_loss: 0.1308 - val_mse: 0.1308
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0240 - mse: 0.0240 - val_loss: 0.1317 - val_mse: 0.1317
Epoch 66/150
33/33 [==============================] - 0s 970us/step - loss: 0.0229 - mse: 0.0229 - val_loss: 0.1311 - val_mse: 0.1311
Epoch 67/150
33/33 [==============================] - 0s 939us/step - loss: 0.0230 - mse: 0.0230 - val_loss: 0.1333 - val_mse: 0.1333
Epoch 68/150
33/33 [==============================] - 0s 939us/step - loss: 0.0227 - mse: 0.0227 - val_loss: 0.1300 - val_mse: 0.1300
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0222 - mse: 0.0222 - val_loss: 0.1303 - val_mse: 0.1303
Epoch 70/150
33/33 [==============================] - 0s 978us/step - 

33/33 [==============================] - 0s 2ms/step - loss: 0.0122 - mse: 0.0122 - val_loss: 0.1323 - val_mse: 0.1323
Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0122 - mse: 0.0122 - val_loss: 0.1316 - val_mse: 0.1316
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0119 - mse: 0.0119 - val_loss: 0.1312 - val_mse: 0.1312
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0119 - mse: 0.0119 - val_loss: 0.1315 - val_mse: 0.1315
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1318 - val_mse: 0.1318
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0114 - mse: 0.0114 - val_loss: 0.1320 - val_mse: 0.1320
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.1318 - val_mse: 0.1318
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0115 -

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [13]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 909us/step - loss: 0.0093 - mse: 0.0093


[0.009283515624701977, 0.009283515624701977]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [14]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 667us/step - loss: 0.1317 - mse: 0.1317


[0.13171051442623138, 0.13171051442623138]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [15]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

28519.489787655803

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [16]:
np.random.seed(123)
he_model = Sequential()

he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))


Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: 0.5522 - mse: 0.5522 - val_loss: 0.2409 - val_mse: 0.2409
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.3015 - mse: 0.3015 - val_loss: 0.1904 - val_mse: 0.1904
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2310 - mse: 0.2310 - val_loss: 0.1703 - val_mse: 0.1703
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2045 - mse: 0.2045 - val_loss: 0.1503 - val_mse: 0.1503
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1731 - mse: 0.1731 - val_loss: 0.1426 - val_mse: 0.1426
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1611 - mse: 0.1611 - val_loss: 0.1397 - val_mse: 0.1397
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1487 - mse: 0.1487 - val_loss: 0.1294 - val_mse: 0.1294
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1433 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0263 - mse: 0.0263 - val_loss: 0.1436 - val_mse: 0.1436
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0269 - mse: 0.0269 - val_loss: 0.1401 - val_mse: 0.1401
Epoch 65/150
33/33 [==============================] - 0s 909us/step - loss: 0.0255 - mse: 0.0255 - val_loss: 0.1417 - val_mse: 0.1417
Epoch 66/150
33/33 [==============================] - 0s 939us/step - loss: 0.0253 - mse: 0.0253 - val_loss: 0.1431 - val_mse: 0.1431
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0250 - mse: 0.0250 - val_loss: 0.1399 - val_mse: 0.1399
Epoch 68/150
33/33 [==============================] - 0s 982us/step - loss: 0.0262 - mse: 0.0262 - val_loss: 0.1423 - val_mse: 0.1423
Epoch 69/150
33/33 [==============================] - 0s 818us/step - loss: 0.0246 - mse: 0.0246 - val_loss: 0.1429 - val_mse: 0.1429
Epoch 70/150
33/33 [==============================] - 0s 818us/step - loss: 0.024

33/33 [==============================] - 0s 939us/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1473 - val_mse: 0.1473
Epoch 125/150
33/33 [==============================] - 0s 909us/step - loss: 0.0118 - mse: 0.0118 - val_loss: 0.1491 - val_mse: 0.1491
Epoch 126/150
33/33 [==============================] - 0s 970us/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.1489 - val_mse: 0.1489
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.1496 - val_mse: 0.1496
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0114 - mse: 0.0114 - val_loss: 0.1474 - val_mse: 0.1474
Epoch 129/150
33/33 [==============================] - 0s 926us/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1473 - val_mse: 0.1473
Epoch 130/150
33/33 [==============================] - 0s 847us/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1479 - val_mse: 0.1479
Epoch 131/150
33/33 [==============================] - 0s 903us/step - lo

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [17]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0087 - mse: 0.0087


[0.008650878444314003, 0.008650878444314003]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [18]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 862us/step - loss: 0.1484 - mse: 0.1484


[0.14839531481266022, 0.14839531481266022]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [20]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=n_features))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: 0.5740 - mse: 0.5740 - val_loss: 0.2385 - val_mse: 0.2385
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2536 - mse: 0.2536 - val_loss: 0.1670 - val_mse: 0.1670
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2001 - mse: 0.2001 - val_loss: 0.1381 - val_mse: 0.1381
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1720 - mse: 0.1720 - val_loss: 0.1258 - val_mse: 0.1258
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1524 - mse: 0.1524 - val_loss: 0.1239 - val_mse: 0.1239
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1392 - mse: 0.1392 - val_loss: 0.1223 - val_mse: 0.1223
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1282 - mse: 0.1282 - val_loss: 0.1224 - val_mse: 0.1224
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1181 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0263 - mse: 0.0263 - val_loss: 0.1294 - val_mse: 0.1294
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0266 - mse: 0.0266 - val_loss: 0.1299 - val_mse: 0.1299
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0257 - mse: 0.0257 - val_loss: 0.1300 - val_mse: 0.1300
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0246 - mse: 0.0246 - val_loss: 0.1294 - val_mse: 0.1294
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0247 - mse: 0.0247 - val_loss: 0.1293 - val_mse: 0.1293
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0246 - mse: 0.0246 - val_loss: 0.1297 - val_mse: 0.1297
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0243 - mse: 0.0243 - val_loss: 0.1302 - val_mse: 0.1302
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0239 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0131 - mse: 0.0131 - val_loss: 0.1367 - val_mse: 0.1367
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0126 - mse: 0.0126 - val_loss: 0.1370 - val_mse: 0.1370
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0126 - mse: 0.0126 - val_loss: 0.1369 - val_mse: 0.1369
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0124 - mse: 0.0124 - val_loss: 0.1364 - val_mse: 0.1364
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0124 - mse: 0.0124 - val_loss: 0.1382 - val_mse: 0.1382
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0122 - mse: 0.0122 - val_loss: 0.1375 - val_mse: 0.1375
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0122 - mse: 0.0122 - val_loss: 0.1375 - val_mse: 0.1375
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0100 - mse: 0.0100


[0.009985336102545261, 0.009985336102545261]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 899us/step - loss: 0.1406 - mse: 0.1406


[0.14057935774326324, 0.14057935774326324]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [23]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
rmsprop_model.compile(optimizer='rmsprop', 
                      loss='mse', 
                      metrics=['mse'])
# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 3ms/step - loss: 0.3552 - mse: 0.3552 - val_loss: 0.1457 - val_mse: 0.1457
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1717 - mse: 0.1717 - val_loss: 0.1389 - val_mse: 0.1389
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1325 - mse: 0.1325 - val_loss: 0.1351 - val_mse: 0.1351
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1072 - mse: 0.1072 - val_loss: 0.1122 - val_mse: 0.1122
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0914 - mse: 0.0914 - val_loss: 0.1169 - val_mse: 0.1169
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0819 - mse: 0.0819 - val_loss: 0.1065 - val_mse: 0.1065
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0601 - mse: 0.0601 - val_loss: 0.1427 - val_mse: 0.1427
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0553 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1143 - val_mse: 0.1143
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0076 - mse: 0.0076 - val_loss: 0.1111 - val_mse: 0.1111
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0084 - mse: 0.0084 - val_loss: 0.1208 - val_mse: 0.1208
Epoch 66/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0070 - mse: 0.0070 - val_loss: 0.1268 - val_mse: 0.1268
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0074 - mse: 0.0074 - val_loss: 0.1171 - val_mse: 0.1171
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0085 - mse: 0.0085 - val_loss: 0.1185 - val_mse: 0.1185
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0084 - mse: 0.0084 - val_loss: 0.1077 - val_mse: 0.1077
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0093 - mse: 0

33/33 [==============================] - 0s 4ms/step - loss: 0.0053 - mse: 0.0053 - val_loss: 0.1277 - val_mse: 0.1277
Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0055 - mse: 0.0055 - val_loss: 0.1163 - val_mse: 0.1163
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.1187 - val_mse: 0.1187
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0055 - mse: 0.0055 - val_loss: 0.1058 - val_mse: 0.1058
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0061 - mse: 0.0061 - val_loss: 0.1349 - val_mse: 0.1349
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.1086 - val_mse: 0.1086
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0047 - mse: 0.0047 - val_loss: 0.1134 - val_mse: 0.1134
Epoch 131/150
33/33 [==============================] - 0s 949us/step - loss: 0.0044

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0027 - mse: 0.0027


[0.002711209002882242, 0.002711209002882242]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [25]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.1215 - mse: 0.1215


[0.12154536694288254, 0.12154536694288254]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [26]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
adam_model.compile(optimizer='Adam', 
                   loss='mse', 
                   metrics=['mse'])
# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.3798 - mse: 0.3798 - val_loss: 0.1174 - val_mse: 0.1174
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1616 - mse: 0.1616 - val_loss: 0.1004 - val_mse: 0.1004
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1143 - mse: 0.1143 - val_loss: 0.0890 - val_mse: 0.0890
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0824 - mse: 0.0824 - val_loss: 0.0907 - val_mse: 0.0907
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0648 - mse: 0.0648 - val_loss: 0.0924 - val_mse: 0.0924
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0472 - mse: 0.0472 - val_loss: 0.0943 - val_mse: 0.0943
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0411 - mse: 0.0411 - val_loss: 0.0997 - val_mse: 0.0997
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0314 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0074 - mse: 0.0074 - val_loss: 0.1015 - val_mse: 0.1015
Epoch 63/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0092 - mse: 0.0092 - val_loss: 0.1122 - val_mse: 0.1122
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0223 - mse: 0.0223 - val_loss: 0.1068 - val_mse: 0.1068
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0140 - mse: 0.0140 - val_loss: 0.1214 - val_mse: 0.1214
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0156 - mse: 0.0156 - val_loss: 0.1166 - val_mse: 0.1166
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0208 - mse: 0.0208 - val_loss: 0.1245 - val_mse: 0.1245
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1007 - val_mse: 0.1007
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0097 - mse: 0

33/33 [==============================] - 0s 3ms/step - loss: 0.0013 - mse: 0.0013 - val_loss: 0.1024 - val_mse: 0.1024
Epoch 123/150
33/33 [==============================] - 0s 3ms/step - loss: 9.6030e-04 - mse: 9.6030e-04 - val_loss: 0.1035 - val_mse: 0.1035
Epoch 124/150
33/33 [==============================] - 0s 4ms/step - loss: 7.4099e-04 - mse: 7.4099e-04 - val_loss: 0.1029 - val_mse: 0.1029
Epoch 125/150
33/33 [==============================] - 0s 6ms/step - loss: 5.3692e-04 - mse: 5.3692e-04 - val_loss: 0.1031 - val_mse: 0.1031
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 5.4596e-04 - mse: 5.4596e-04 - val_loss: 0.1049 - val_mse: 0.1049
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 5.7786e-04 - mse: 5.7786e-04 - val_loss: 0.1039 - val_mse: 0.1039
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 4.8380e-04 - mse: 4.8380e-04 - val_loss: 0.1025 - val_mse: 0.1025
Epoch 129/150
33/33 [==============

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [27]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 727us/step - loss: 0.0028 - mse: 0.0028


[0.002828782657161355, 0.002828782657161355]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [28]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 782us/step - loss: 0.1163 - mse: 0.1163


[0.11632716655731201, 0.11632716655731201]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [29]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 793us/step - loss: 0.1618 - mse: 0.1618


[0.16176044940948486, 0.16176044940948486]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [30]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))


31605.863855475007

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.